In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](01_tensorqs_tutorial.ipynb) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| [Build
Model](buildmodel_tutorial.html) \|\|
[Autograd](autogradqs_tutorial.html) \|\| **Optimization** \|\| [Save &
Load Model](saveloadrun_tutorial.html)

Optimizing Model Parameters
===========================

Now that we have a model and data it\'s time to train, validate and test
our model by optimizing its parameters on our data. Training a model is
an iterative process; in each iteration the model makes a guess about
the output, calculates the error in its guess (*loss*), collects the
derivatives of the error with respect to its parameters (as we saw in
the [previous section](autograd_tutorial.html)), and **optimizes** these
parameters using gradient descent. For a more detailed walkthrough of
this process, check out this video on [backpropagation from
3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

Prerequisite Code
-----------------

We load the code from the previous sections on [Datasets &
DataLoaders](data_tutorial.html) and [Build
Model](buildmodel_tutorial.html).

---

**모델 매개변수 최적화**  
===========================  

이제 모델과 데이터가 준비되었으므로, 데이터를 기반으로 모델을 훈련하고 검증하며 테스트하여 매개변수를 최적화할 차례입니다. 모델 훈련은 반복적인 과정입니다. 각 반복에서 모델은 출력에 대한 추측을 하고, 그 추측의 오류(*손실*)를 계산하며, 매개변수에 대한 오류의 미분(기울기)을 수집합니다(이전 섹션에서 설명한 바와 같이). 이후 **경사 하강법**(gradient descent)을 사용하여 이러한 매개변수를 최적화합니다. 이 과정에 대한 자세한 설명은 [3Blue1Brown의 역전파(backpropagation) 동영상](https://www.youtube.com/watch?v=tIeHLnjs5U8)을 참조하세요.

**선행 코드**  
-----------------  

이전 섹션에서 다룬 [데이터셋 및 데이터 로더](data_tutorial.html)와 [모델 구축](buildmodel_tutorial.html) 코드를 로드합니다.


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Hyperparameters
===============

Hyperparameters are adjustable parameters that let you control the model
optimization process. Different hyperparameter values can impact model
training and convergence rates ([read
more](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)
about hyperparameter tuning)

We define the following hyperparameters for training:

:   -   **Number of Epochs** - the number times to iterate over the
        dataset
    -   **Batch Size** - the number of data samples propagated through
        the network before the parameters are updated
    -   **Learning Rate** - how much to update models parameters at each
        batch/epoch. Smaller values yield slow learning speed, while
        large values may result in unpredictable behavior during
        training.

---

**하이퍼파라미터**  
===============  

하이퍼파라미터는 모델 최적화 과정을 제어할 수 있게 해주는 조정 가능한 매개변수입니다. 서로 다른 하이퍼파라미터 값은 모델 훈련과 수렴 속도에 영향을 미칠 수 있습니다 ([하이퍼파라미터 튜닝에 대해 더 알아보기](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)).

훈련을 위한 다음 하이퍼파라미터를 정의합니다:

- **에폭 수 (Number of Epochs)** - 데이터셋을 반복하는 횟수
- **배치 크기 (Batch Size)** - 매개변수가 업데이트되기 전에 네트워크를 통해 전파되는 데이터 샘플의 수
- **학습률 (Learning Rate)** - 각 배치/에폭마다 모델 매개변수를 얼마나 업데이트할 것인지. 작은 값은 느린 학습 속도를 초래하며, 큰 값은 훈련 중 예측할 수 없는 행동을 유발할 수 있습니다.

In [2]:
learning_rate = 1e-3 # 1 * 10^-3
batch_size = 64
epochs = 5

Optimization Loop
=================

Once we set our hyperparameters, we can then train and optimize our
model with an optimization loop. Each iteration of the optimization loop
is called an **epoch**.

Each epoch consists of two main parts:

:   -   **The Train Loop** - iterate over the training dataset and try
        to converge to optimal parameters.
    -   **The Validation/Test Loop** - iterate over the test dataset to
        check if model performance is improving.

Let\'s briefly familiarize ourselves with some of the concepts used in
the training loop. Jump ahead to see the
`full-impl-label`{.interpreted-text role="ref"} of the optimization
loop.

Loss Function
-------------

When presented with some training data, our untrained network is likely
not to give the correct answer. **Loss function** measures the degree of
dissimilarity of obtained result to the target value, and it is the loss
function that we want to minimize during training. To calculate the loss
we make a prediction using the inputs of our given data sample and
compare it against the true data label value.

Common loss functions include
[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)
(Mean Square Error) for regression tasks, and
[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)
(Negative Log Likelihood) for classification.
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)
combines `nn.LogSoftmax` and `nn.NLLLoss`.

We pass our model\'s output logits to `nn.CrossEntropyLoss`, which will
normalize the logits and compute the prediction error.

---

**최적화 루프**  
=================

하이퍼파라미터를 설정한 후, 최적화 루프를 통해 모델을 훈련하고 최적화할 수 있습니다. 최적화 루프의 각 반복을 **에폭**(epoch)이라고 합니다.

각 에폭은 두 가지 주요 부분으로 구성됩니다:

- **훈련 루프 (The Train Loop)** - 훈련 데이터셋을 반복하며 최적의 매개변수에 수렴하려고 시도합니다.
- **검증/테스트 루프 (The Validation/Test Loop)** - 테스트 데이터셋을 반복하여 모델 성능이 향상되고 있는지 확인합니다.

훈련 루프에서 사용되는 몇 가지 개념에 대해 간단히 알아보겠습니다. 최적화 루프의 `full-impl-label`{.interpreted-text role="ref"}로 바로 가실 수 있습니다.

**손실 함수 (Loss Function)**  
-------------

훈련 데이터를 제공받았을 때, 훈련되지 않은 네트워크는 올바른 답을 제공하지 않을 가능성이 높습니다. **손실 함수**는 얻어진 결과와 목표 값 간의 비유사성을 측정하며, 우리는 훈련 중에 이 손실 함수를 최소화하고자 합니다. 손실을 계산하기 위해 주어진 데이터 샘플의 입력을 사용하여 예측을 수행하고, 이를 실제 데이터 레이블 값과 비교합니다.

일반적인 손실 함수로는 회귀 작업에 사용되는 [nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (평균 제곱 오차)와 분류 작업에 사용되는 [nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (음의 로그 우도)가 있습니다. [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)는 `nn.LogSoftmax`와 `nn.NLLLoss`를 결합한 것입니다.

모델의 출력 로짓(logits)을 `nn.CrossEntropyLoss`에 전달하면, 로짓이 정규화되고 예측 오류가 계산됩니다.

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

Optimizer
=========

Optimization is the process of adjusting model parameters to reduce
model error in each training step. **Optimization algorithms** define
how this process is performed (in this example we use Stochastic
Gradient Descent). All optimization logic is encapsulated in the
`optimizer` object. Here, we use the SGD optimizer; additionally, there
are many [different
optimizers](https://pytorch.org/docs/stable/optim.html) available in
PyTorch such as ADAM and RMSProp, that work better for different kinds
of models and data.

We initialize the optimizer by registering the model\'s parameters that
need to be trained, and passing in the learning rate hyperparameter.

---

**최적화기 (Optimizer)**  
=========  

최적화는 각 훈련 단계에서 모델 오류를 줄이기 위해 모델 매개변수를 조정하는 과정입니다. **최적화 알고리즘**은 이 과정이 어떻게 수행되는지를 정의합니다(이 예제에서는 확률적 경사 하강법(Stochastic Gradient Descent, SGD)을 사용합니다). 모든 최적화 로직은 `optimizer` 객체에 캡슐화되어 있습니다. 여기서는 SGD 최적화를 사용하며, PyTorch에는 ADAM, RMSProp 등과 같이 다양한 [최적화기](https://pytorch.org/docs/stable/optim.html)가 있어, 모델과 데이터 유형에 따라 더 잘 작동하는 경우가 많습니다.

최적화기는 훈련해야 할 모델의 매개변수를 등록하고, 학습률 하이퍼파라미터를 전달하여 초기화합니다.

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:

:   -   Call `optimizer.zero_grad()` to reset the gradients of model
        parameters. Gradients by default add up; to prevent
        double-counting, we explicitly zero them at each iteration.
    -   Backpropagate the prediction loss with a call to
        `loss.backward()`. PyTorch deposits the gradients of the loss
        w.r.t. each parameter.
    -   Once we have our gradients, we call `optimizer.step()` to adjust
        the parameters by the gradients collected in the backward pass.

---

훈련 루프 내에서 최적화는 세 가지 단계로 진행됩니다:

- **`optimizer.zero_grad()` 호출:** 모델 매개변수의 기울기를 초기화합니다. 기울기는 기본적으로 누적되기 때문에, 이중 계산을 방지하기 위해 각 반복에서 명시적으로 0으로 설정합니다.
  
- **손실 역전파:** `loss.backward()`를 호출하여 예측 손실을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 기울기를 저장합니다.

- **매개변수 조정:** 기울기를 확보한 후, `optimizer.step()`을 호출하여 역전파에서 수집된 기울기에 따라 매개변수를 조정합니다.

Full Implementation {#full-impl-label}
===================

We define `train_loop` that loops over our optimization code, and
`test_loop` that evaluates the model\'s performance against our test
data.

**전체 구현** {#full-impl-label}  
===================  

`train_loop`을 정의하여 최적화 코드를 반복하고, `test_loop`를 정의하여 테스트 데이터에 대한 모델의 성능을 평가합니다.

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    # 모델을 training node로 설정 - batch normalization과 dropout layers를 위해 중요
    # 지금 상황에선 필수적이지 않지만 최선의 연습을 위해 추가
    model.train()

    for batch, (X, y) in enumerate(dataloader): # 데이터 로더에서 배치 단위로 반복
        pred = model(X) # 입력 X에 대한 예측
        loss = loss_fn(pred, y) # 예측과 실제 값 y 간의 손실 계산

        # BackPropagation
        loss.backward() # 손실에 대한 기울기 계산
        optimizer.step() # Optimizer를 사용하여 Parameters 업데이트
        optimizer.zero_grad() # 기울기 0으로 초기화

        # 100번째 배치마다 손실 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [10]:
def test_loop(dataloader, model, loss_fn):

    # 모델을 평가 모드로 설정 - normalization과 dropout layers를 위해 중요
    # 지금 상황에는 필요하지 않지만 최선의 연습을 위해 추가
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # `torch.no_grad()`를 사용하여 모델을 평가하면 테스트 모드에서 기울기가 계산되지 않음
    # 이는 `requires_grad = True`인 텐서에 대한 불 필요한 기울기 계산과 메모리 사용을 줄이는 데에도 도움이 됩니다.
    with torch.no_grad():
        for X, y in dataloader: 
            pred = model(X)
            test_loss += loss_fn(pred, y).item() # 손실을 누적
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # 정확도 계산

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to
`train_loop` and `test_loop`. Feel free to increase the number of epochs
to track the model\'s improving performance.

---

손실 함수와 최적화기를 초기화한 후, 이를 `train_loop`와 `test_loop`에 전달합니다. 에폭 수를 늘려 모델의 성능 향상을 추적해 보세요.

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300094 [   64/60000]
loss: 2.290082 [ 6464/60000]
loss: 2.271015 [12864/60000]
loss: 2.266727 [19264/60000]
loss: 2.248260 [25664/60000]
loss: 2.224128 [32064/60000]
loss: 2.225450 [38464/60000]
loss: 2.190200 [44864/60000]
loss: 2.180830 [51264/60000]
loss: 2.167274 [57664/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 2.146781 

Epoch 2
-------------------------------
loss: 2.150030 [   64/60000]
loss: 2.141581 [ 6464/60000]
loss: 2.078056 [12864/60000]
loss: 2.100466 [19264/60000]
loss: 2.047601 [25664/60000]
loss: 1.989044 [32064/60000]
loss: 2.014000 [38464/60000]
loss: 1.930779 [44864/60000]
loss: 1.928427 [51264/60000]
loss: 1.878672 [57664/60000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.859762 

Epoch 3
-------------------------------
loss: 1.886954 [   64/60000]
loss: 1.859519 [ 6464/60000]
loss: 1.732371 [12864/60000]
loss: 1.786255 [19264/60000]
loss: 1.681132 [25664/60000]
loss: 1.634204 [32064/60000]
loss: 1.660297 [38464/

Further Reading
===============

-   [Loss
    Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
-   [torch.optim](https://pytorch.org/docs/stable/optim.html)
-   [Warmstart Training a
    Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)
